<a href="https://colab.research.google.com/github/wenyenyeh/HW5-Deep-Learning-Basic/blob/main/HW5_Deep_Learningbasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 安裝 PyTorch 和 PyTorch Lightning
!pip install torch torchvision pytorch-lightning

# 安裝 TensorFlow (如果尚未安裝)
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 50.2 MB/s eta 0:00:00


HW5-1: Iris Classification Problem
使用 tf.keras:

In [6]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import numpy as np

# 加載Iris數據集
iris = load_iris()
X = iris.data
y = iris.target

# 數據預處理
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = tf.keras.utils.to_categorical(y, 3)  # One-hot encoding

# 訓練集和測試集分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 建立模型
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.2),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

# 定義早期停止和學習率調度器
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)

# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping, lr_scheduler])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 987ms/step - accuracy: 0.4263 - loss: 1.2737 - val_accuracy: 0.8000 - val_loss: 0.8734 - learning_rate: 0.0010
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.4948 - loss: 1.0554 - val_accuracy: 0.8333 - val_loss: 0.8121 - learning_rate: 9.5000e-04
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.6021 - loss: 0.8900 - val_accuracy: 0.8667 - val_loss: 0.7610 - learning_rate: 9.0250e-04
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6925 - loss: 0.7406 - val_accuracy: 0.9000 - val_loss: 0.7169 - learning_rate: 8.5737e-04
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7083 - loss: 0.6610 - val_accuracy: 0.9000 - val_loss: 0.6815 - learning_rate: 8.1451e-04
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8052 - loss: 0.5525 - val_accuracy: 0.9000 - val_loss: 0.6512 - learning_rate: 7.7378e-04
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7906 - loss: 0.

HW5-1: Iris Classification Problem 使用 PyTorch:

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import LabelEncoder

# 加載數據
iris = load_iris()
X = iris.data
y = iris.target

# 數據預處理
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# 訓練集和測試集分割
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# 定義模型
class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)
        self.dropout = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = torch.relu(self.batch_norm1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.batch_norm2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.softmax(x, dim=1)

# 定義訓練過程
model = IrisModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.95)

# 訓練模型
epochs = 100
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    scheduler.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}/{epochs}, Loss: {loss.item()}')


Epoch 0/100, Loss: 1.0893903970718384
Epoch 10/100, Loss: 0.8107530474662781
Epoch 20/100, Loss: 0.682647705078125
Epoch 30/100, Loss: 0.6150875687599182
Epoch 40/100, Loss: 0.6299540996551514
Epoch 50/100, Loss: 0.6871748566627502
Epoch 60/100, Loss: 0.6215670704841614
Epoch 70/100, Loss: 0.6531760096549988
Epoch 80/100, Loss: 0.5875367522239685
Epoch 90/100, Loss: 0.6347842812538147


HW5-1: Iris Classification Problem 使用 PyTorch Lightning:

In [8]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.optim import Adam
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

# 加載數據
iris = load_iris()
X = iris.data
y = iris.target

# 數據預處理
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# 訓練集和測試集分割
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# 定義模型
class IrisLightningModel(pl.LightningModule):
    def __init__(self):
        super(IrisLightningModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)
        self.dropout = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.batch_norm2 = nn.BatchNorm1d(32)

    def forward(self, x):
        x = torch.relu(self.batch_norm1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.batch_norm2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return torch.softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        return loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=0.001)
        scheduler = StepLR(optimizer, step_size=10, gamma=0.95)
        return [optimizer], [scheduler]

# 訓練模型
model = IrisLightningModel()
trainer = pl.Trainer(max_epochs=100)
trainer.fit(model, train_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type        | Params | Mode 
----------------------------------------------------
0 | fc1         | Linear      | 320    | train
1 | fc2         | Linear      | 2.1 K  | train
2 | fc3         | Linear      | 99     | train
3 | dropout     | Dropout     | 0      | train
4 | batch_norm1 | BatchNorm1d | 128    | train
5 | batch_norm2 | BatchNorm1d | 64     | train
----------------------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval m

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


HW5-2: Handwritten Digit Recognition (MNIST)

使用 tf.keras:

In [9]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# 加載數據
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 數據預處理
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 建立 CNN 模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

# 訓練模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, callbacks=[early_stopping, lr_scheduler])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.8709 - loss: 0.4218 - val_accuracy: 0.9859 - val_loss: 0.0435 - learning_rate: 0.0010
Epoch 2/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9814 - loss: 0.0604 - val_accuracy: 0.9882 - val_loss: 0.0338 - learning_rate: 9.5000e-04
Epoch 3/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9863 - loss: 0.0413 - val_accuracy: 0.9889 - val_loss: 0.0329 - learning_rate: 9.0250e-04
Epoch 4/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9910 - loss: 0.0288 - val_accuracy: 0.9911 - val_loss: 0.0258 - learning_rate: 8.5737e-04
Epoch 5/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9918 - loss: 0.0248 - val_accuracy: 0.9914 - val_loss: 0.0258 - learning_rate: 8.1451e-04
Epoch 6/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9935 - loss: 0.0202 - val_accuracy: 0.9908 - val_loss: 0.0276 - learning_rate: 7.7378e-04
Epoch 7/20
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.

使用 PyTorch:

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 數據加載和預處理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = DataLoader(testset, batch_size=64)

# 定義 CNN 模型
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64*5*5, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64*5*5)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 訓練模型
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.18MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 65.6kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:06<00:00, 244kB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.18MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Loss: 0.18136729300022125
Epoch 2, Loss: 0.1422044336795807
Epoch 3, Loss: 0.0014197974232956767
Epoch 4, Loss: 0.0701633170247078
Epoch 5, Loss: 0.06702471524477005
Epoch 6, Loss: 0.023593179881572723
Epoch 7, Loss: 0.1012800857424736
Epoch 8, Loss: 0.01048573013395071
Epoch 9, Loss: 0.002079547615721822
Epoch 10, Loss: 0.01088274922221899


HW5-3: CIFAR Image Classification (VGG19 Pretrained)

使用 tf.keras:

In [12]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# 加載CIFAR-10數據
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 數據預處理
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 使用VGG19預訓練模型
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 添加自定義層
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

# 訓練模型
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64, callbacks=[early_stopping, lr_scheduler])


Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.4465 - loss: 1.5702 - val_accuracy: 0.5516 - val_loss: 1.2762 - learning_rate: 0.0010
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.5619 - loss: 1.2538 - val_accuracy: 0.5653 - val_loss: 1.2285 - learning_rate: 9.5000e-04
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5856 - loss: 1.1832 - val_accuracy: 0.5818 - val_loss: 1.2017 - learning_rate: 9.0250e-04
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.6019 - loss: 1.1244 - val_accuracy: 0.5847 - val_loss: 1.1711 - learning_rate: 8.5737e-04
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.6195 - loss: 1.0824 - val_accuracy: 0.5905 - val_loss: 1.1722 - learning_rate: 8.1451e-04
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 15ms/step - accuracy: 0.6280 - loss: 1.0577 - val_accuracy: 0.5949 - val_loss: 1.1571 - learning_rate: 7.7378e-04
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - 